In [1]:
%load_ext jupyter_black

# nvidia cuda


``` bash
nvidia-smi --query-gpu=timestamp,name,pci.bus_id,driver_version,pstate,pcie.link.gen.max,pcie.link.gen.current,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv
nvidia-smi --query-gpu=name,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv -l 5

nvidia-smi -q -g 0 -d UTILIZATION -l
==============NVSMI LOG==============

Timestamp                                 : Sat Aug 13 18:07:02 2022
Driver Version                            : 516.59
CUDA Version                              : 11.7

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Utilization
        Gpu                               : 1 %
        Memory                            : 1 %
        Encoder                           : 0 %
        Decoder                           : 0 %
    GPU Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 6 %
        Min                               : 0 %
        Avg                               : 0 %
    Memory Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 1 %
        Min                               : 0 %
        Avg                               : 0 %
    ENC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
    DEC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
```

In [2]:
import distributed

from dask_cuda import LocalCUDACluster


def close():
    """
    for convenience if a cluster/client is already running
    this should close them to prevent starting multiple clusters
    on the same port.
    """
    try:
        client.close(), cluster.close()
    except:
        ...


close()
cluster = LocalCUDACluster()
client = distributed.Client(cluster)

client

/opt/conda/envs/rapids/lib/python3.9/site-packages/dask_cuda/utils.py:244: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
2022-09-02 00:47:08,268 - distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/griblib/notebooks/probsevere/dask-worker-space/worker-53hgbka3', purging
2022-09-02 00:47:08,269 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 15.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39729,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 15.57 GiB
Comm: tcp://127.0.0.1:39911,Total threads: 1
Dashboard: http://127.0.0.1:33557/status,Memory: 15.57 GiB
Nanny: tcp://127.0.0.1:34479,


In [3]:
import os
from pathlib import Path
import pandas as pd
import dask.dataframe as dd
import cudf

# creating a Path object from the absolute path to our data folder
ps_data = Path.cwd().parents[1] / "data-bucket"
all_files = tuple(ps_data.rglob("*.pq"))
ps_data

PosixPath('/workspaces/griblib/data-bucket')

In [4]:
len(all_files)

8612

In [5]:
%%time
# reading with pure pandas and pyarrow
pd.concat((pd.read_parquet(file, engine="pyarrow") for file in all_files))

CPU times: user 51.9 s, sys: 12.9 s, total: 1min 4s
Wall time: 49.3 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
3,2066,1657,-1,42.099998,284,39.099998,0.29,1.94,7,0.26,...,5.374,26,111651,2022-04-16 08:30:35,-89.620003,34.200001,-89.550003,34.270000,-89.584999,34.230000
3,1078,285,-38,37.900002,123,23.600000,0.03,0.88,0,0.00,...,-0.975,3,113455,2022-04-16 14:24:40,-84.000000,32.040001,-83.779999,32.230000,-83.925682,32.150002
3,678,251,-160,48.299999,87,18.500000,0.06,0.95,1,0.06,...,5.381,6,112748,2022-04-16 11:46:37,-85.440002,31.100000,-85.139999,31.280001,-85.230415,31.195000
3,27,0,0,8.400000,212,33.099998,0.00,0.39,0,0.00,...,2.418,0,112144,2022-04-16 09:38:38,-82.930000,36.689999,-82.779999,36.790001,-82.855003,36.735001
3,206,0,0,21.500000,343,37.099998,0.00,0.65,1,0.06,...,-2.198,0,110971,2022-04-16 05:52:39,-86.059998,35.150002,-85.720001,35.509998,-85.894997,35.330002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,1599,0,-999,26.600000,406,36.099998,0.00,0.53,1,0.03,...,-9.219,1,85839,2022-05-08 06:02:41,-98.050003,41.889999,-97.970001,41.970001,-98.010002,41.924999
52,1534,1194,-999,32.400002,29,19.299999,0.07,0.54,0,0.00,...,5.095,3,86072,2022-05-08 07:04:36,-75.599998,33.410000,-75.489998,33.459999,-75.537498,33.439999
52,1092,0,0,48.299999,287,41.099998,0.00,0.56,0,0.00,...,-5.997,5,88941,2022-05-08 18:32:34,-93.239998,42.590000,-93.180000,42.650002,-93.209999,42.625000


In [6]:
%%time
# much faster to read into a dask dataframe
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf

CPU times: user 472 ms, sys: 22.3 ms, total: 495 ms
Wall time: 475 ms


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,MAXLLAZ,P98LLAZ,P98MLAZ,WETBULB_0C_HGT,PWAT,CAPE_M10M30,LJA,SIZE,MOTION_EAST,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
npartitions=8612,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,uint32,int32,float32,uint32,float32,float32,float32,uint32,float32,float32,float32,float32,float32,float32,uint32,float32,uint32,float32,float32,uint8,uint32,datetime64[ns],float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
%%time
# again, but the compute time to convert it back to a pandas native object is more expensive
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf.compute()

CPU times: user 31 s, sys: 2.12 s, total: 33.1 s
Wall time: 1min 21s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
0,1524,1270,0,51.400002,36,17.900000,0.92,2.08,36,1.25,...,6.203,88,136524,2022-03-01 00:00:29,-79.209999,30.120001,-79.070000,30.280001,-79.139999,30.200001
0,61,74,0,11.100000,225,46.000000,0.00,0.45,0,0.00,...,2.775,0,137200,2022-03-01 10:42:34,-122.099998,45.380001,-121.949997,45.509998,-122.023331,45.450001
0,74,76,0,12.200000,227,45.599998,0.00,0.45,0,0.00,...,-0.083,0,137200,2022-03-01 10:40:35,-122.169998,45.330002,-121.949997,45.500000,-122.055496,45.410000
0,990,642,0,32.299999,27,13.300000,0.10,0.70,0,0.00,...,-0.477,2,137196,2022-03-01 10:38:37,-78.800003,26.600000,-78.709999,26.709999,-78.756248,26.655001
0,990,642,0,32.299999,27,13.300000,0.11,0.72,0,0.00,...,38.986,2,137196,2022-03-01 10:36:32,-78.800003,26.600000,-78.709999,26.709999,-78.756248,26.655001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2745,1591,-9,33.000000,44,8.800000,0.00,0.99,0,0.00,...,1.656,3,212549,2022-06-05 22:32:40,-80.379997,26.280001,-80.300003,26.379999,-80.349998,26.334999
175,2282,1917,-5,23.000000,0,3.000000,0.41,1.59,4,0.19,...,-1.140,3,212377,2022-06-05 22:12:40,-85.849998,31.549999,-85.779999,31.620001,-85.815002,31.580000
176,2700,2077,-8,26.299999,0,2.300000,0.10,0.81,0,0.00,...,1.686,3,212378,2022-06-05 22:12:40,-83.959999,30.049999,-83.889999,30.139999,-83.925003,30.090000
176,2218,1669,-1,13.200000,35,13.900000,0.00,0.83,0,0.00,...,0.811,0,212550,2022-06-05 22:32:40,-78.910004,24.959999,-78.839996,25.040001,-78.883331,25.000000


In [8]:
%%time
# the parquet files we're create using dask.dataframe
gdf: cudf.DataFrame = cudf.read_parquet(all_files)
df = gdf.to_pandas()
del gdf  # delete the dataframe from gpu memory after it has been loaded to gpu ram
df

CPU times: user 5.81 s, sys: 4.54 s, total: 10.4 s
Wall time: 10.4 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,X,Y
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
3,2066,1657,-1,42.099998,284,39.099998,0.29,1.94,7,0.26,...,5.374,26,111651,2022-04-16 08:30:35,-89.620003,34.200001,-89.550003,34.270000,-89.584999,34.230000
3,1078,285,-38,37.900002,123,23.600000,0.03,0.88,0,0.00,...,-0.975,3,113455,2022-04-16 14:24:40,-84.000000,32.040001,-83.779999,32.230000,-83.925682,32.150002
3,678,251,-160,48.299999,87,18.500000,0.06,0.95,1,0.06,...,5.381,6,112748,2022-04-16 11:46:37,-85.440002,31.100000,-85.139999,31.280001,-85.230415,31.195000
3,27,0,0,8.400000,212,33.099998,0.00,0.39,0,0.00,...,2.418,0,112144,2022-04-16 09:38:38,-82.930000,36.689999,-82.779999,36.790001,-82.855003,36.735001
3,206,0,0,21.500000,343,37.099998,0.00,0.65,1,0.06,...,-2.198,0,110971,2022-04-16 05:52:39,-86.059998,35.150002,-85.720001,35.509998,-85.894997,35.330002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,1599,0,-999,26.600000,406,36.099998,0.00,0.53,1,0.03,...,-9.219,1,85839,2022-05-08 06:02:41,-98.050003,41.889999,-97.970001,41.970001,-98.010002,41.924999
52,1534,1194,-999,32.400002,29,19.299999,0.07,0.54,0,0.00,...,5.095,3,86072,2022-05-08 07:04:36,-75.599998,33.410000,-75.489998,33.459999,-75.537498,33.439999
52,1092,0,0,48.299999,287,41.099998,0.00,0.56,0,0.00,...,-5.997,5,88941,2022-05-08 18:32:34,-93.239998,42.590000,-93.180000,42.650002,-93.209999,42.625000


In [9]:
%%time
from contextlib import contextmanager
from typing import Iterable, Generator

# the parquet files we're create using dask.dataframe
@contextmanager
def gpu_session(files: Iterable[Path]) -> Generator[cudf.DataFrame, None, None]:
    """
    context manager to remove the cudf from menory
    """
    try:
        gdf = cudf.read_parquet(files).set_index(["VALIDTIME", "ID"]).sort_index()
        yield gdf
    finally:
        del gdf


with gpu_session(all_files) as gdf:
    df = gdf.to_pandas()

df

CPU times: user 4.61 s, sys: 2.15 s, total: 6.76 s
Wall time: 6.8 s


MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
VALIDTIME           ID                                                  
2022-03-01 00:00:29 136524    1524    1270      0  51.400002       36   
                    136565     756     433      0  39.200001       82   
                    136592     384     112     -5  37.900002       80   
                    136648     551     422     -4  33.299999       24   
                    136649      10       0   -999   6.400000      170   
...                            ...     ...    ...        ...      ...   
2022-06-05 23:58:37 213271    2944    2318     -4  25.700001       35   
                    213272    3099    2506     -6  29.700001        7   
                    213273    3094    2577     -2  29.000000       20   
                    213274    1732    1419     -1  42.200001       40   
                    213275    2792    2075   -999  15.100000       17   

                            MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  FLASH_RATE  \
VALIDTIME           ID                                                         
2022-03-01 00:00:29 136524          17.900000  0.92         2.08          36   
                    136565          16.200001  0.11         0.68           0   
                    136592           7.300000  0.00         0.78           0   
                    136648          18.000000  0.08         0.94           0   
                    136649          39.000000  0.00         0.27           0   
...                                       ...   ...          ...         ...   
2022-06-05 23:58:37 213271           7.000000  0.00         0.62           0   
                    213272           0.800000  0.08         0.73           0   
                    213273           1.200000  0.08         1.01           1   
                    213274           6.000000  0.00         0.36           0   
                    213275          22.700001  0.07         0.22           0   

                            FLASH_DENSITY  ...  SIZE  MOTION_EAST  \
VALIDTIME           ID                     ...                      
2022-03-01 00:00:29 136524           1.25  ...   201        6.493   
                    136565           0.00  ...    55        8.784   
                    136592           0.00  ...   271        5.375   
                    136648           0.02  ...    83        6.686   
                    136649           0.00  ...   119        5.171   
...                                   ...  ...   ...          ...   
2022-06-05 23:58:37 213271           0.00  ...    96        7.871   
                    213272           0.00  ...    52        5.973   
                    213273           0.06  ...    43        5.286   
                    213274           0.00  ...    48       28.112   
                    213275           0.02  ...    78       28.112   

                            MOTION_SOUTH  PS        MINX       MINY  \
VALIDTIME           ID                                                
2022-03-01 00:00:29 136524         6.203  88  -79.209999  30.120001   
                    136565         5.193   4  -80.320000  28.020000   
                    136592         5.236   2  -80.949997  29.990000   
                    136648         5.148   2  -81.129997  28.309999   
                    136649         4.775   0 -123.760002  47.470001   
...                                  ...  ..         ...        ...   
2022-06-05 23:58:37 213271         0.459   1  -93.220001  32.770000   
                    213272         0.371   3  -91.980003  30.549999   
                    213273         0.704   3  -92.059998  30.320000   
                    213274        -0.172   4  -79.949997  28.920000   
                    213275        -0.172   1  -76.379997  25.230000   

                                  MAXX       MAXY           X          Y  
VALIDTIME           ID                                                    
2022-03-01 00:00:29 136524  -79.070000  30.280001  -79.139999  30

In [27]:
df.size == 118_772_271

True